In [1]:
import os
import sys

from tqdm.notebook import tqdm
from sentiment_utils import clean_mentions, get_batches, apply_sentiment_analysis, convert_to_list, update_sentiment_scores

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "_0_Constants_and_Utils"))

from database_utils import get_dataframe_from_query, form_connection_params


c:\My programs\Python 3.12\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [2]:
local = False
connection_params = form_connection_params(local, True)
query = """
SELECT tweet_id, full_text
FROM Tweets
WHERE sentiment_score IS NULL;
"""
batch_size = 1_000

In [3]:
test_data = get_dataframe_from_query(query, connection_params, local, index_col="tweet_id")

In [4]:
# Apply the cleaning function to the DataFrame
test_data['cleaned_text'] = test_data['full_text'].apply(clean_mentions)

In [5]:
test_data

,full_text,cleaned_text
tweet_id,,
1000021306509856768,"In this week's Ryanair #Euronews, we go behind...","In this week's Ryanair #Euronews, we go behind..."
1000091345036423169,Not quiet a wing but close enough. @KLM_UK @K...,Not quiet a wing but close enough. 737 appr...
1000134458098507778,Singapore Airlines SQ406 taxing towards runway...,Singapore Airlines SQ406 taxing towards runway...
1000319540977393664,.@VirginAtlantic great service before my fligh...,. great service before my flight in the lounge...
1000360896005398528,Another 747 @lufthansa @BoeingAirplanes B747...,Another 747 B747-430 D-ABVX operating flig...
...,...,...
999631569328443392,Whites flying to Paris get $5000. Black Disabl...,Whites flying to Paris get $5000. Black Disabl...
999682962030546944,"Last winter we de-iced over 6,000 aircraft, wi...","Last winter we de-iced over 6,000 aircraft, wi..."
999688090884165632,"@KLM Toch een fout van Twitter. Dat ze via ""u...","Toch een fout van Twitter. Dat ze via ""uitgel..."


In [6]:
data_batches = get_batches(test_data[["cleaned_text"]], batch_size)

In [7]:
for batch in tqdm(data_batches, desc="Updating sentiment_score: "):
    # feel free to change the 
    df_sentiment = apply_sentiment_analysis(batch, "cleaned_text", 128, 10)
    update_sentiment_scores(convert_to_list(df_sentiment), connection_params, local)

Updating sentiment_score:   0%|          | 0/6149 [00:00<?, ?it/s]

KeyboardInterrupt: 